In [1]:
%tensorflow_version 2.x
import tensorflow as tf
!pip install -U sentence-transformers
!pip install gensim
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec 
from google.colab import drive
import json
!pip install --upgrade ibm-watson
from ibm_watson import AssistantV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
import itertools
import nltk
nltk.download('punkt')
import re
import math
import glob
import os
!pip install python-docx
from gensim.models import Doc2Vec
import docx
import time
import gc
import torch
from sentence_transformers import SentenceTransformer, util

drive.mount('/content/drive')
iamAutentikator = 'Jljl7cVGjh5Jmm9E8OHYI1iu7xmNSoGOyUaZqtBwewUG'
workspaceID = 'e13262d3-e6e1-4ad5-b460-9561ffefbdb1'
assistantURL = 'https://api.us-south.assistant.watson.cloud.ibm.com'

authenticator = IAMAuthenticator(iamAutentikator)
assistant = AssistantV1(
    version='2020-04-01',
    authenticator = authenticator
)

assistant.set_service_url(assistantURL)

listDatas = []

# model = FastText.load('idwiki_word2vec_200.model')
# model = KeyedVectors.load_word2vec_format('cc.id.300.vec')


def createIntent(question, ans, examp):
  
  intent_in = str(question).replace(" ", "")
  more_examp = " ".join(ans)
  response=assistant.create_intent(
      workspace_id=workspaceID,
      intent=intent_in,
      examples=[
          {'text': examp[0]},
          {'text': examp[1]},
          {'text': examp[2]},
          {'text': examp[3]},
          {'text': examp[4]},
          {'text': examp[5]},
          {'text': examp[6]},
          {'text': examp[7]},
          {'text': examp[8]},
          {'text': examp[9]},
          {'text': question},
         # {'text': more_examp}
          
      ]
  ).get_result()

  del examp
  del intent_in
  del more_examp
  del response
  gc.collect()

def createDialog(question, ans):
    intent_in = str(question).replace(" ", "")
    con = "#" + intent_in
    answer = list2string(ans)
    response=assistant.create_dialog_node(
        workspace_id=workspaceID,
        dialog_node=question,
        conditions=con,
        output={
            'generic': [
                {
                    'response_type': 'text',
                    'values': [
                        {
                            'text': answer
                        }
                    ]
                }
            ]
        }
    ).get_result()

    del con
    del answer
    del intent_in
    del response
    gc.collect()


def updateIntent(question, ans, examp):
    # examp = makeExamp(question)
    intent_in = str(question).replace(" ", "")
    more_examp = " ".join(ans)
    # print(more_examp)
    response=assistant.update_intent(
        workspace_id=workspaceID,
        intent=intent_in,
        new_examples=[
            {'text': examp[0]},
            {'text': examp[1]},
            {'text': examp[2]},
            {'text': examp[3]},
            {'text': examp[4]},
            {'text': examp[5]},
            {'text': examp[6]},
            {'text': examp[7]},
            {'text': examp[8]},
            {'text': examp[9]},
            {'text': question},
           # {'text': more_examp}
        ]   
    ).get_result

    del examp
    del intent_in
    del more_examp
    del response
    # gc.collect()

def updateDialog(question, ans):
    intent_in = str(question).replace(" ", "")
    con = '#' + intent_in
    answer = list2string(ans)
    response=assistant.update_dialog_node(
        workspace_id=workspaceID,
        dialog_node=question,
        conditions=con,
        new_output={
            'generic': [
                {
                    'response_type': 'text',
                    'values': [
                        {
                            'text': answer
                        }
                    ]
                }
            ]
        }
    ).get_result()

    del con
    del answer
    del intent_in
    del response
    # gc.collect()
    # print(json.dumps(response, indent=2))

def list2string(s):
    str1 = '\n'

    return str1.join(s)

def getdoc():
  path = '/content/drive/My Drive/Colab Notebooks/DOKUMEN/'
    # array     
    
  for filename in glob.glob(os.path.join(path, '*.docx')):
    files = docx.Document(filename)
    ans = []
    # for para in files.paragraphs:
    #     teks.append(para.text)
    
    question = files.paragraphs[0].text
    question = re.sub('[^A-Za-z0-9]+',' ', question)

    for i in range(1, len(files.paragraphs)):
        para = files.paragraphs[i].text
        ans.append(para)

    examp = makeExamp(question)
    # print(ans)
    # print(" ".join(ans))
    try:
        # question = str(question).replace(" ", "")
        # print(question)
        
        createIntent(question, ans, examp)
        
    except:
        # question = str(question).replace(" ", "")
        
        updateIntent(question, ans, examp)
    
    try:
        
        createDialog(question, ans)
        
    except:
        
        updateDialog(question, ans)
    
    
    gc.collect()
    

def makeExamp(kalimat):
  kalimat = kalimat.lower()
  words = kalimat.split()
  sim_words = []

  kal_vec = model_bert.encode(kalimat, convert_to_tensor=True)

  for word in words:
    temp = []
    try:
      sim_word = model.wv.most_similar(positive=word, topn=2)
      for each in sim_word:
        temp.append(each[0])
      
    except:
      temp.append(word)
    sim_words.append(temp)
    
  

  sentence = list(itertools.product(*sim_words))
  del sim_words
  del words
  

  corp = []  
  #   sentence.append(sen)
  for each in sentence:
    temp = ' '.join(each)
    corp.append(temp)
    
  
  del sentence

  # examp = []
  #vecs = []
  #important

  # kal_vec = doc_model.wv.get_vector(kalimat)

  # for each_sentence in corp:
    
  #   vecs.append(doc_model.wv.get_vector(each_sentence))
  vecs = model_bert.encode(corp, convert_to_tensor=True)

  cos_sim = util.pytorch_cos_sim(kal_vec, vecs)[0]
    
  # cos_sim = model.wv.cosine_similarities(kal_vec, vecs)
  
    

  # k = 0
  # for i in corp:
  #   temp = (cos_sim[k], i)
  #   k = k+1
  #   examp.append(temp)

  # p_sort = sorted(examp, reverse=True)

  # res = p_sort[:10]
  # res = corp[:10]
  res = []
  cos_sim = cos_sim.cpu()
  top_res = torch.topk(cos_sim, k=10)

  for score, idx in zip(top_res[0], top_res[1]):
    res.append(corp[idx])
    print(corp[idx], "(Score: %.4f)" % (score))

  # del examp
  del corp
  # del p_sort
  del kal_vec
  del cos_sim
  # del k
  del vecs
  
  gc.collect()
  

  return res

# name = input()
model = Word2Vec.load('/content/drive/My Drive/Colab Notebooks/Model/idwiki_word2vec_200_new_lower.model')
# # fast_model = FastText.load_facebook_vectors('/content/drive/My Drive/Colab Notebooks/Model/cc.id.300.vec')
# doc_model = Doc2Vec.load('/content/drive/My Drive/Colab Notebooks/Model/idwiki_word2vec_200_new_lower.model')
model_bert = SentenceTransformer('distiluse-base-multilingual-cased')
getdoc()
# makeExamp('apakah sudah ada obat untuk covid19')

Requirement already up-to-date: sentence-transformers in /usr/local/lib/python3.6/dist-packages (0.3.8)
Requirement already up-to-date: ibm-watson in /usr/local/lib/python3.6/dist-packages (4.7.1)
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:252: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


metode melindunginya dirinya apabila tak menyadari siapakah tersebut tertular covid 19 (Score: 0.8495)
metode melindunginya lugosi apabila tak menyadari siapakah tersebut tertular covid 19 (Score: 0.8472)
metode melindunginya dirinya bila tak menyadari siapakah tersebut tertular covid 19 (Score: 0.8364)
metode melindunginya dirinya apabila tak menyadari siapakah tersebut disuntik covid 19 (Score: 0.8359)
metode melindunginya lugosi apabila belum menyadari siapakah tersebut tertular covid 19 (Score: 0.8346)
metode melindunginya dirinya apabila belum menyadari siapakah tersebut tertular covid 19 (Score: 0.8344)
metode melindunginya dirinya apabila tak menyadari apa tersebut tertular covid 19 (Score: 0.8309)
caranya melindunginya dirinya apabila tak menyadari siapakah tersebut tertular covid 19 (Score: 0.8309)
metode melindunginya lugosi apabila tak menyadari siapakah tersebut disuntik covid 19 (Score: 0.8306)
metode melindunginya dirinya bila belum menyadari siapakah tersebut tertular co